## This is a regression problem, im thinking of using linear regression for this dataset, ill try to get a little in depth more... maybe use ridge,lasso and elastic as well

### Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import probplot
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression,Ridge, Lasso, ElasticNet, RidgeCV, LassoCV,ElasticNetCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Download data from here
# https://www.kaggle.com/ahmettezcantekin/beginner-datasets

df = pd.read_csv("../input/beginner-datasets/beginner_datasets/diamond.csv")

## EDA

In [ ]:
df.head()

In [ ]:
df.info()
# looks like no null values

In [ ]:
df.describe()

In [ ]:
for col in df.columns:
    if df[col].dtype == 'object':
        print(col,": ",df[col].unique())

#### lets try to find a relationship between the independent categorical variables and price

In [ ]:
df.groupby('Cut').describe()

# i think cut is a ordinal variable, cut matters

In [ ]:
df.groupby('Color').describe()
# color might be nominal... there is a variation but its not huge

In [ ]:
df.groupby('Clarity').describe()

# to be frank, for now i don't know if i can properly interpret this way so i'll use some plots which will make it easy

# we can also just use target guided ordinal encoding so we'll look into that too... (replace with means)

### Internet says:
#### 1.boxplots
#### 2.Jittered plots (no idea what it is)
#### 3.violin plots ( finally!, a usecase)

In [ ]:
# it is jittered i guess

for col in df.columns:
    if df[col].dtype == 'object':
        sns.set_theme(style="whitegrid")
        ax = sns.stripplot(x=col, y="Price", data=df);
        plt.show()

# i feel like my eda is getting better...
# practice more is the value to go!

### Good! this is actually helpful, but keep in mind in the real world such relationships can be hard to get... like if you have less data it will be hard to understand where it falls (eg: FL in clarity column). 

### The way data is collected is also important... not all samples are good predictor of the population distribution, some can be really bad 

In [ ]:
# boxplots

for col in df.columns:
    if df[col].dtype == 'object':
        sns.set_theme(style="whitegrid")
        ax = sns.boxplot(df[col], df['Price']);
        plt.show()

In [ ]:
# violin plots

for col in df.columns:
    if df[col].dtype == 'object':
        sns.set_theme(style="whitegrid")
        ax = sns.violinplot(df['Price'], df[col],oreint='h');
        plt.show()
        
# from seaborn:-
# A violin plot plays a similar role as a box and whisker plot. It shows the distribution of quantitative data across several 
# levels of one (or more) categorical variables such that those distributions can be compared. Unlike a box plot, in which all 
# of the plot components correspond to actual datapoints, the violin plot features a kernel density estimation of the
# underlying distribution.

## Feature engineering

### Now, lets use this to encode our categorical data

In [ ]:
#ok we'll try target guided ordinal encoding first

# we'll use the describe method to get the values... i was gonna write it down but i realized i can make a dictionary directly

cut_vals = df.groupby('Cut').describe()['Price']['mean'].to_dict()
color_vals = df.groupby('Color').describe()['Price']['mean'].to_dict()
clarity_vals = df.groupby('Clarity').describe()['Price']['mean'].to_dict()
polish_vals = df.groupby('Polish').describe()['Price']['mean'].to_dict()
symmetry_vals = df.groupby('Symmetry').describe()['Price']['mean'].to_dict()
report_vals = df.groupby('Report').describe()['Price']['mean'].to_dict()

In [ ]:
# replace all... i used as int type because floats take more space and i dont think that will make a difference

df['tg_'+'cut'] = df['Cut'].replace(cut_vals).astype('int')
df['tg_'+'color'] = df['Color'].replace(color_vals).astype('int')
df['tg_'+'clarity'] = df['Clarity'].replace(clarity_vals).astype('int')
df['tg_'+'polish'] = df['Polish'].replace(polish_vals).astype('int')
df['tg_'+'symmetry'] = df['Symmetry'].replace(symmetry_vals).astype('int')
df['tg_'+'report'] = df['Report'].replace(report_vals).astype('int')

In [ ]:
# lets use a heat map to see the correlation

cor = np.round(df.corr(),2)
plt.figure(figsize=(6,6))
sns.heatmap(cor, annot=True, cmap='Greens');

In [ ]:
# checking others out of curiosity... 

# for i in ['min','max','50%','25%','75%']:
#     cut_vals = df.groupby('Cut').describe()['Price'][i].to_dict()
#     color_vals = df.groupby('Color').describe()['Price'][i].to_dict()
#     clarity_vals = df.groupby('Clarity').describe()['Price'][i].to_dict()
#     polish_vals = df.groupby('Polish').describe()['Price'][i].to_dict()
#     symmetry_vals = df.groupby('Symmetry').describe()['Price'][i].to_dict()
#     report_vals = df.groupby('Report').describe()['Price'][i].to_dict()

#     df['tg_'+'cut'] = df['Cut'].replace(cut_vals).astype('int')
#     df['tg_'+'color'] = df['Color'].replace(color_vals).astype('int')
#     df['tg_'+'clarity'] = df['Clarity'].replace(clarity_vals).astype('int')
#     df['tg_'+'polish'] = df['Polish'].replace(polish_vals).astype('int')
#     df['tg_'+'symmetry'] = df['Symmetry'].replace(symmetry_vals).astype('int')
#     df['tg_'+'report'] = df['Report'].replace(report_vals).astype('int')

#     cor = np.round(df.corr(),2)
#     plt.figure(figsize=(12,12))
#     sns.heatmap(cor, annot=True, cmap='Greens');
#     plt.show();

In [ ]:
# tried to get these from the graphs, but they didnt work out that well though similar to mean

# cut_p  = {'Ideal':5, 'Very Good':4, 'Fair':1, 'Good':3, 'Signature-Ideal':2}
# color_p =  {'H':2, 'E':5, 'G':3,'D':6,'F':4, 'I':1}
# clarity_p =  {'SI1':1, 'VS1':3 ,'VS2':2, 'VVS2':5 ,'VVS1':4 ,'IF':7 ,'FL':6}
# polish_p =  {'VG':3 ,'ID':1, 'EX':4, 'G':2}
# symmetry_p =  {'EX':4, 'ID':1, 'VG':3, 'G':2}
# report_p =  {'GIA':2, 'AGSL':1}

# df['p_'+'cut'] = df['Cut'].replace(cut_p).astype('int')
# df['p_'+'color'] = df['Color'].replace(color_p).astype('int')
# df['p_'+'clarity'] = df['Clarity'].replace(clarity_p).astype('int')
# df['p_'+'polish'] = df['Polish'].replace(polish_p).astype('int')
# df['p_'+'symmetry'] = df['Symmetry'].replace(symmetry_p).astype('int')
# df['p_'+'report'] = df['Report'].replace(report_p).astype('int')

# cor = np.round(df.corr(),2)
# plt.figure(figsize=(10,10))
# sns.heatmap(cor, annot=True, cmap='Greens');

### looks like carat weight, cut, color, clarity matter are top most important

In [ ]:
# ax1 = plt.subplot(221)
probplot(df['Carat Weight'], plot=plt);

#### WOW!

## lets fix that

## tried a few transformations but they don't make a desirable difference

In [ ]:
probplot(np.power(df['Carat Weight'],3), plot=plt);

In [ ]:
sns.distplot(df['Carat Weight']);

### i'm gonna use this

https://towardsdatascience.com/an-easy-tool-to-correctly-transform-non-linear-data-for-linear-regression-5fbe7f7bfe2f

### wait... we can't, its for bulging data which we dont have but worth noting... check above link for more info

In [ ]:
# maybe linear regression won't do so well here...

In [ ]:
# lets use standard scaler

X = df.drop(['Price','Cut','Color','Clarity','Polish','Symmetry','Report'], axis=1)
y = df.Price


scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

variables = X_scaled

# we create a new data frame which will include all the VIFs
# note that each variable has its own variance inflation factor as this measure is variable specific (not model specific)
# we do not include categorical values for mulitcollinearity as they do not provide much information as numerical ones do
vif = pd.DataFrame()

# here we make use of the variance_inflation_factor, which will basically output the respective VIFs 
vif["VIF"] = [variance_inflation_factor(variables, i) for i in range(variables.shape[1])]
# Finally, I like to include names so it is easier to explore the result
vif["Features"] = X.columns
vif

## Feature selection

### we'll go with carat weight, cut, color, clarity (target encoded)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
model = LinearRegression()
model.fit(X_train,y_train)
y_pred = model.predict(X_test)
model.score(X_test,y_test)

In [ ]:
print(cross_val_score(model, X, y , cv=5))
np.mean(cross_val_score(model, X, y , cv=5))

### Not bad...

In [ ]:
ridge = Ridge()
parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,1,5,10,20,30,35,40,45,50,55,90,100,110]}
ridge_regressor = GridSearchCV(ridge,parameters,scoring='neg_mean_squared_error', cv=5)
ridge_regressor.fit(X,y)

In [ ]:
print(ridge_regressor.best_params_)
print(ridge_regressor.best_score_)

In [ ]:
ridge = Ridge(alpha=0.01)
ridge.fit(X_train,y_train)
ridge.score(X_test,y_test)

In [ ]:
lasso = Lasso()
parameters = {'alpha':[1e-15,1e-10,1e-8,1e-3,1e-2,0.1,1,5,10,20,30,35,40,45,50]}
lasso_regressor = GridSearchCV(lasso,parameters,scoring='neg_mean_squared_error', cv=5)
lasso_regressor.fit(X,y)
                               
    
print(lasso_regressor.best_params_)
print(lasso_regressor.best_score_)

In [ ]:
lasso = Lasso(alpha=0.1)
lasso.fit(X_train,y_train)
lasso.score(X_test,y_test)

### Now, their is a thing that when it comes to linear regression we assume the error rate is normally distributed...

In [ ]:
prediction_lasso = lasso.predict(X_test)
prediction_ridge = ridge.predict(X_test)

In [ ]:
sns.distplot(y_test-prediction_lasso);   # right skewed

In [ ]:
sns.distplot(y_test-prediction_ridge);   # right skewed

In [ ]:
# Lasso Regularization
# LassoCV will return best alpha and coefficients after performing 10 cross validations
lasscv = LassoCV(alphas = None,cv =10, max_iter = 100000, normalize = True)
lasscv.fit(X_train, y_train)

# best alpha parameter
alpha = lasscv.alpha_
alpha

In [ ]:
lasso_reg = Lasso(alpha)
lasso_reg.fit(X_train, y_train)
lasso_reg.score(X_test, y_test)

In [ ]:
# Using Ridge regression model
# RidgeCV will return best alpha and coefficients after performing 10 cross validations. 
# We will pass an array of random numbers for ridgeCV to select best alpha from them

alphas = np.random.uniform(low=0, high=10, size=(50,))
ridgecv = RidgeCV(alphas = alphas,cv=10,normalize = True)
ridgecv.fit(X_train, y_train)

ridgecv.alpha_

In [ ]:
ridge_model = Ridge(alpha=ridgecv.alpha_)
ridge_model.fit(X_train, y_train)

In [ ]:
ridge_model.score(X_test, y_test)

In [ ]:
elasticCV = ElasticNetCV(alphas = None, cv =10)

elasticCV.fit(X_train, y_train)

In [ ]:
elasticCV.alpha_

In [ ]:
# l1_ration gives how close the model is to L1 regularization, below value indicates we are giving equal
#preference to L1 and L2
elasticCV.l1_ratio

In [ ]:
elasticnet_reg = ElasticNet(alpha = elasticCV.alpha_,l1_ratio=0.5)
elasticnet_reg.fit(X_train, y_train)
elasticnet_reg.score(X_test, y_test)

## Performance metrics

In [ ]:
model = LinearRegression()
mse = cross_val_score(model,X,y,scoring='neg_mean_squared_error', cv=5)
mean_mse = np.mean(mse)
print(mse)
print(mean_mse)

#### These metrics would be good for comparing models but simple linear regression itself is performing the best in our case so we'll compare it with elastic net

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
# root mean squared error
np.sqrt(mean_squared_error(y_test, y_pred))

# Thank you and don't forget to give a like/star if it was worth your time